In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense, LSTM, TimeDistributed, GRU, SimpleRNN
from keras.models import Model
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

In [3]:
data = pd.read_csv("./LOBs_Benchmark/processed_data.csv")
data.head()

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level5_Bid_Price,level5_Bid_Volume,level5_Ask_Price,level5_Ask_Volume,Quoted_spread,Weighted_Mid_Price,Mid_Price,Next_mid,Price_move,up_or_down
0,0.093,269.0,2.0,290.0,3.0,268.0,2.0,485.0,2.0,258.0,...,251.0,5.0,780.0,0.0,21.0,277.4,279.5,279.5,0.0,1
1,0.062,269.0,2.0,290.0,3.0,268.0,2.0,358.0,4.0,258.0,...,251.0,5.0,1142.0,0.0,21.0,277.4,279.5,279.5,0.0,1
2,0.031,269.0,2.0,290.0,3.0,268.0,2.0,322.0,4.0,258.0,...,251.0,5.0,739.0,0.0,21.0,277.4,279.5,279.5,0.0,1
3,0.031,269.0,2.0,290.0,3.0,268.0,2.0,322.0,4.0,258.0,...,6.0,4.0,739.0,0.0,21.0,277.4,279.5,279.5,0.0,1
4,0.155,269.0,2.0,290.0,3.0,268.0,2.0,322.0,4.0,263.0,...,6.0,4.0,739.0,0.0,21.0,277.4,279.5,285.0,5.5,1


In [4]:
from sklearn.model_selection import train_test_split
data=data.astype(np.float32)
df_input=data.drop(columns=['Mid_Price'])
df_target=data['Mid_Price']
df_target = np.expand_dims(df_target, axis=1)
X_train,X_test, y_train, y_test = train_test_split(df_input, df_target,test_size=0.2,shuffle=False)
print(len(X_train), len(y_train), len(X_test), len(y_test))

152954 152954 38239 38239


AttributeError: 'numpy.ndarray' object has no attribute 'tail'

In [5]:
data.describe()

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level5_Bid_Price,level5_Bid_Volume,level5_Ask_Price,level5_Ask_Volume,Quoted_spread,Weighted_Mid_Price,Mid_Price,Next_mid,Price_move,up_or_down
count,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,...,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000,191193.000000
mean,0.122387,264.055786,4.748563,307.688416,2.906749,254.020386,5.015680,354.511230,3.020215,234.059525,...,162.974976,3.951525,706.814758,1.982923,43.632561,288.151093,285.872131,285.872040,-0.000065,0.760007
std,0.113098,24.843811,4.159198,67.235092,1.481309,40.298462,4.314088,113.166138,1.514673,62.167511,...,94.568069,3.967979,272.905762,1.860757,67.408920,43.995937,37.852146,37.852169,15.328211,0.426803
min,0.031000,4.000000,1.000000,232.000000,1.000000,1.000000,0.000000,233.000000,0.000000,0.001942,...,0.000905,0.000000,240.000000,0.000000,1.000000,80.000000,135.000000,135.000000,-268.500000,0.000000
25%,0.031000,254.000000,2.000000,271.000000,2.000000,250.000000,2.000000,280.000000,2.000000,228.000000,...,79.000000,1.000000,528.000000,0.000000,4.000000,266.000000,265.500000,265.500000,0.000000,1.000000
50%,0.093000,270.000000,3.000000,286.000000,3.000000,266.000000,4.000000,302.000000,3.000000,258.000000,...,179.000000,3.000000,677.000000,2.000000,15.000000,281.333344,281.000000,281.000000,0.000000,1.000000
75%,0.155000,280.000000,6.000000,315.000000,4.000000,277.000000,7.000000,395.000000,4.000000,274.000000,...,252.000000,5.000000,792.000000,4.000000,52.000000,294.750000,295.000000,295.000000,0.000000,1.000000
max,2.015000,293.000000,39.000000,800.000000,12.000000,291.000000,41.000000,960.000000,13.000000,289.000000,...,287.000000,38.000000,1893.000000,15.000000,647.000000,760.230774,541.500000,541.500000,253.500000,1.000000


In [6]:
from sklearn.preprocessing import StandardScaler
feature_scaler=StandardScaler()
X_train=feature_scaler.fit_transform(X_train)
X_test=feature_scaler.transform(X_test)

target_scaler=StandardScaler()
y_train=target_scaler.fit_transform(y_train)
y_test=target_scaler.transform(y_test)

In [7]:
def reshape_to_batches(a, batch_size):
    #pad if the length is not divisible by the batch_size
    batch_num = np.ceil(a.shape[0] / float(batch_size))
    modulo = batch_num * batch_size - a.shape[0]
    if modulo != 0:
        pad = np.zeros((int(modulo), a.shape[1]))
        a = np.vstack((a, pad))
    return np.array(np.split(a, batch_num))

In [8]:
def reshape_to_batches(a, batch_size):
    #pad if the length is not divisible by the batch_size
    batch_num = np.ceil(a.shape[0] / float(batch_size))
    modulo = batch_num * batch_size - a.shape[0]
    if modulo != 0:
        pad = np.zeros((int(modulo), a.shape[1]))
        a = np.vstack((a, pad))
    return np.array(np.split(a, batch_num))

In [9]:
batch_size = 30 #30 is about 165 minute (5.5min per ts x 30), tried 15 and a few other values too but 30 seems to be the best

X_train_batch = reshape_to_batches(X_train, batch_size)
y_train_batch = reshape_to_batches(y_train, batch_size)
print(X_train_batch.shape, y_train_batch.shape)

X_test_batch = reshape_to_batches(X_test, batch_size)
y_test_batch = reshape_to_batches(y_test, batch_size)
print(X_test_batch.shape, y_test_batch.shape)

(5099, 30, 26) (5099, 30, 1)
(1275, 30, 26) (1275, 30, 1)


In [10]:
# don't count the Mid-price in
def df_to_time_step(df_x,df_y, window_size=30):
  # df_x_np = df_x.to_numpy()
  # df_y_np = df_y.to_numpy()
  X = []
  y = []
  for i in range(len(df_x)-window_size):
    row = [r for r in df_x[i:i+window_size]] # the whole row
    X.append(row)
    label = df_y[i+window_size] # temperature
    y.append(label)
  return np.array(X), np.array(y)

In [11]:
timestep=100
X_train_step, y_train_step=df_to_time_step(X_train, y_train,timestep)
X_test_step, y_test_step= df_to_time_step(X_test,y_test,timestep)
print(X_train_step.shape, y_train_step.shape)
print(X_test_step.shape, y_test_step.shape)

(152854, 100, 26) (152854, 1)
(38139, 100, 26) (38139, 1)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [13]:
#Allows the network to accept batches of any size as input when predicting
# Make the model
model1 = Sequential()
lstm=LSTM(32,activation='tanh',recurrent_activation='hard_sigmoid', use_bias=True,
            kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal',
            bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None,
            recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None,
            kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0,
            recurrent_dropout=0.0, implementation=1, return_sequences=True, return_state=False,
            go_backwards=False, stateful=False, unroll=False, input_shape=(batch_size, 26))
model1.add(lstm)
model1.add(TimeDistributed(Dense(1, activation='linear')))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 32)            7552      
                                                                 
 time_distributed (TimeDistr  (None, 30, 1)            33        
 ibuted)                                                         
                                                                 
Total params: 7,585
Trainable params: 7,585
Non-trainable params: 0
_________________________________________________________________


In [14]:
model1.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [15]:
model1_fit=model1.fit(X_train_batch, y_train_batch, epochs=500, batch_size=100)

Epoch 1/500
51/51 [==============================] - 4s 25ms/step - loss: 0.3432 - accuracy: 1.0460e-04
Epoch 2/500
51/51 [==============================] - 1s 25ms/step - loss: 0.0670 - accuracy: 1.0460e-04
Epoch 3/500
51/51 [==============================] - 1s 24ms/step - loss: 0.0214 - accuracy: 1.0460e-04
Epoch 4/500
51/51 [==============================] - 1s 24ms/step - loss: 0.0099 - accuracy: 1.0460e-04
Epoch 5/500
51/51 [==============================] - 1s 24ms/step - loss: 0.0058 - accuracy: 1.0460e-04
Epoch 6/500
51/51 [==============================] - 1s 24ms/step - loss: 0.0040 - accuracy: 1.0460e-04
Epoch 7/500
51/51 [==============================] - 1s 25ms/step - loss: 0.0031 - accuracy: 1.0460e-04
Epoch 8/500
51/51 [==============================] - 1s 25ms/step - loss: 0.0024 - accuracy: 1.0460e-04
Epoch 9/500
51/51 [==============================] - 1s 25ms/step - loss: 0.0021 - accuracy: 1.0460e-04
Epoch 10/500
51/51 [==============================] - 1s 25ms/st

51/51 [==============================] - 1s 24ms/step - loss: 4.2213e-04 - accuracy: 1.0460e-04
Epoch 77/500
51/51 [==============================] - 1s 24ms/step - loss: 4.4573e-04 - accuracy: 1.0460e-04
Epoch 78/500
51/51 [==============================] - 1s 24ms/step - loss: 4.2722e-04 - accuracy: 1.0460e-04
Epoch 79/500
51/51 [==============================] - 1s 24ms/step - loss: 4.3561e-04 - accuracy: 1.0460e-04
Epoch 80/500
51/51 [==============================] - 1s 24ms/step - loss: 4.2197e-04 - accuracy: 1.0460e-04
Epoch 81/500
51/51 [==============================] - 1s 24ms/step - loss: 4.0636e-04 - accuracy: 1.0460e-04
Epoch 82/500
51/51 [==============================] - 1s 24ms/step - loss: 4.4323e-04 - accuracy: 1.0460e-04
Epoch 83/500
51/51 [==============================] - 1s 24ms/step - loss: 4.4825e-04 - accuracy: 1.0460e-04
Epoch 84/500
51/51 [==============================] - 1s 24ms/step - loss: 3.9223e-04 - accuracy: 1.0460e-04
Epoch 85/500
51/51 [============

51/51 [==============================] - 2s 30ms/step - loss: 3.1761e-04 - accuracy: 1.0460e-04
Epoch 151/500
51/51 [==============================] - 2s 30ms/step - loss: 3.1773e-04 - accuracy: 1.0460e-04
Epoch 152/500
51/51 [==============================] - 2s 30ms/step - loss: 3.2949e-04 - accuracy: 1.0460e-04
Epoch 153/500
51/51 [==============================] - 2s 30ms/step - loss: 3.2443e-04 - accuracy: 1.0460e-04
Epoch 154/500
51/51 [==============================] - 2s 30ms/step - loss: 2.8586e-04 - accuracy: 1.0460e-04
Epoch 155/500
51/51 [==============================] - 1s 29ms/step - loss: 3.0711e-04 - accuracy: 1.0460e-04
Epoch 156/500
51/51 [==============================] - 1s 28ms/step - loss: 3.1649e-04 - accuracy: 1.0460e-04
Epoch 157/500
51/51 [==============================] - 1s 28ms/step - loss: 3.3393e-04 - accuracy: 1.0460e-04
Epoch 158/500
51/51 [==============================] - 1s 29ms/step - loss: 2.9011e-04 - accuracy: 1.0460e-04
Epoch 159/500
51/51 [===

51/51 [==============================] - 2s 30ms/step - loss: 2.9338e-04 - accuracy: 1.0460e-04
Epoch 225/500
51/51 [==============================] - 2s 30ms/step - loss: 2.3161e-04 - accuracy: 1.0460e-04
Epoch 226/500
51/51 [==============================] - 2s 30ms/step - loss: 2.7611e-04 - accuracy: 1.0460e-04
Epoch 227/500
51/51 [==============================] - 1s 29ms/step - loss: 2.5476e-04 - accuracy: 1.0460e-04
Epoch 228/500
51/51 [==============================] - 2s 30ms/step - loss: 2.7475e-04 - accuracy: 1.0460e-04
Epoch 229/500
51/51 [==============================] - 2s 30ms/step - loss: 2.7591e-04 - accuracy: 1.0460e-04
Epoch 230/500
51/51 [==============================] - 2s 30ms/step - loss: 2.4204e-04 - accuracy: 1.0460e-04
Epoch 231/500
51/51 [==============================] - 1s 29ms/step - loss: 2.4083e-04 - accuracy: 1.0460e-04
Epoch 232/500
51/51 [==============================] - 2s 30ms/step - loss: 2.7484e-04 - accuracy: 1.0460e-04
Epoch 233/500
51/51 [===

51/51 [==============================] - 2s 30ms/step - loss: 2.1215e-04 - accuracy: 1.0460e-04
Epoch 344/500
51/51 [==============================] - 2s 30ms/step - loss: 2.0263e-04 - accuracy: 1.0460e-04
Epoch 345/500
51/51 [==============================] - 2s 30ms/step - loss: 2.3964e-04 - accuracy: 1.0460e-04
Epoch 346/500
51/51 [==============================] - 2s 30ms/step - loss: 1.7472e-04 - accuracy: 1.0460e-04
Epoch 347/500
51/51 [==============================] - 2s 30ms/step - loss: 2.1139e-04 - accuracy: 1.0460e-04
Epoch 348/500
51/51 [==============================] - 2s 31ms/step - loss: 2.0735e-04 - accuracy: 1.0460e-04
Epoch 349/500
51/51 [==============================] - 2s 30ms/step - loss: 1.9293e-04 - accuracy: 1.0460e-04
Epoch 350/500
51/51 [==============================] - 2s 30ms/step - loss: 2.1620e-04 - accuracy: 1.0460e-04
Epoch 351/500
51/51 [==============================] - 2s 30ms/step - loss: 2.0567e-04 - accuracy: 1.0460e-04
Epoch 352/500
51/51 [===

51/51 [==============================] - 2s 30ms/step - loss: 1.6734e-04 - accuracy: 1.0460e-04
Epoch 418/500
51/51 [==============================] - 1s 29ms/step - loss: 1.8262e-04 - accuracy: 1.0460e-04
Epoch 419/500
51/51 [==============================] - 2s 30ms/step - loss: 2.1130e-04 - accuracy: 1.0460e-04
Epoch 420/500
51/51 [==============================] - 1s 29ms/step - loss: 1.6395e-04 - accuracy: 1.0460e-04
Epoch 421/500
51/51 [==============================] - 2s 30ms/step - loss: 1.9255e-04 - accuracy: 1.0460e-04
Epoch 422/500
51/51 [==============================] - 2s 30ms/step - loss: 1.7904e-04 - accuracy: 1.0460e-04
Epoch 423/500
51/51 [==============================] - 2s 30ms/step - loss: 1.8905e-04 - accuracy: 1.0460e-04
Epoch 424/500
51/51 [==============================] - 2s 30ms/step - loss: 1.8067e-04 - accuracy: 1.0460e-04
Epoch 425/500
51/51 [==============================] - 1s 28ms/step - loss: 1.7727e-04 - accuracy: 1.0460e-04
Epoch 426/500
51/51 [===

51/51 [==============================] - 2s 30ms/step - loss: 1.7826e-04 - accuracy: 1.0460e-04
Epoch 492/500
51/51 [==============================] - 1s 29ms/step - loss: 1.6980e-04 - accuracy: 1.0460e-04
Epoch 493/500
51/51 [==============================] - 2s 30ms/step - loss: 1.7519e-04 - accuracy: 1.0460e-04
Epoch 494/500
51/51 [==============================] - 2s 30ms/step - loss: 1.7210e-04 - accuracy: 1.0460e-04
Epoch 495/500
51/51 [==============================] - 2s 30ms/step - loss: 1.5795e-04 - accuracy: 1.0460e-04
Epoch 496/500
51/51 [==============================] - 1s 29ms/step - loss: 1.7649e-04 - accuracy: 1.0460e-04
Epoch 497/500
51/51 [==============================] - 2s 30ms/step - loss: 1.6353e-04 - accuracy: 1.0460e-04
Epoch 498/500
51/51 [==============================] - 2s 30ms/step - loss: 1.7995e-04 - accuracy: 1.0460e-04
Epoch 499/500
51/51 [==============================] - 1s 28ms/step - loss: 1.6472e-04 - accuracy: 1.0460e-04
Epoch 500/500
51/51 [===

In [16]:
# Predict the values
y_pred=model1.predict(X_test_batch)
print(y_pred.shape)

40/40 [==============================] - 1s 9ms/step
(1275, 30, 1)


In [17]:
#linear
from sklearn.metrics import mean_absolute_percentage_error
y_predict=y_pred.flatten()
y_predict_t=y_predict.reshape(-1,1)

y_test_t=y_test_batch.flatten()
y_test_t1=y_test_t.reshape(-1,1)
y_predict_o=target_scaler.inverse_transform(y_predict_t)
y_test_o=target_scaler.inverse_transform(y_test_t1)
y_predict_o=y_predict_o.flatten()
y_test_o=y_test_o.flatten()
print(mean_absolute_error(y_test_o,y_predict_o))
print(mean_squared_error(y_test_o,y_predict_o))
print(mean_absolute_percentage_error(y_test_o,y_predict_o))
test_results = pd.DataFrame(data={'Test Predictions':y_predict_o, 'Actuals':y_test_o})
test_results

0.22903241358069115
0.19893775929164778
0.0009043865949523832


,Test Predictions,Actuals
0,276.183990,276.500000
1,268.955353,269.000001
2,268.477997,268.500000
3,268.564880,268.500000
4,268.534180,268.500000
...,...,...
38245,289.948242,289.811038
38246,289.942627,289.811038
38247,289.939423,289.811038
38248,289.937653,289.811038


In [40]:
df = data.tail(38239)
df.reset_index(drop=True, inplace=True)
df

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level5_Bid_Price,level5_Bid_Volume,level5_Ask_Price,level5_Ask_Volume,Quoted_spread,Weighted_Mid_Price,Mid_Price,Next_mid,Price_move,up_or_down
0,0.062,257.0,6.0,296.0,5.0,256.0,1.0,306.0,2.0,255.0,...,252.0,5.0,583.0,2.0,39.0,278.272736,276.5,269.0,-7.5,0.0
1,0.062,257.0,6.0,281.0,5.0,256.0,1.0,296.0,5.0,255.0,...,252.0,5.0,583.0,2.0,24.0,270.090912,269.0,268.5,-0.5,0.0
2,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,251.0,3.0,583.0,2.0,23.0,269.545441,268.5,268.5,0.0,1.0
3,0.186,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,251.0,3.0,619.0,2.0,23.0,269.545441,268.5,268.5,0.0,1.0
4,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,255.0,...,252.0,5.0,619.0,2.0,23.0,269.545441,268.5,268.5,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38234,0.186,249.0,8.0,283.0,5.0,246.0,22.0,523.0,3.0,244.0,...,132.0,1.0,795.0,4.0,34.0,269.923065,266.0,265.5,-0.5,0.0
38235,0.124,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,132.0,1.0,795.0,4.0,33.0,269.307678,265.5,265.5,0.0,1.0
38236,0.031,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,132.0,1.0,795.0,4.0,33.0,269.307678,265.5,265.5,0.0,1.0
38237,0.248,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,132.0,1.0,795.0,4.0,33.0,269.307678,265.5,265.0,-0.5,0.0


In [41]:
test_result = test_results[:38239]

In [42]:
test_result

,Test Predictions,Actuals
0,276.183990,276.500000
1,268.955353,269.000001
2,268.477997,268.500000
3,268.564880,268.500000
4,268.534180,268.500000
...,...,...
38234,266.008026,266.000000
38235,265.544495,265.500001
38236,265.541656,265.500001
38237,265.533051,265.500001


In [49]:
new_df = pd.merge(df, test_result, left_index=True, right_index=True)
new_df

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level5_Ask_Price,level5_Ask_Volume,Quoted_spread,Weighted_Mid_Price,Mid_Price,Next_mid,Price_move,up_or_down,Test Predictions,Actuals
0,0.062,257.0,6.0,296.0,5.0,256.0,1.0,306.0,2.0,255.0,...,583.0,2.0,39.0,278.272736,276.5,269.0,-7.5,0.0,276.183990,276.500000
1,0.062,257.0,6.0,281.0,5.0,256.0,1.0,296.0,5.0,255.0,...,583.0,2.0,24.0,270.090912,269.0,268.5,-0.5,0.0,268.955353,269.000001
2,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,583.0,2.0,23.0,269.545441,268.5,268.5,0.0,1.0,268.477997,268.500000
3,0.186,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,619.0,2.0,23.0,269.545441,268.5,268.5,0.0,1.0,268.564880,268.500000
4,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,255.0,...,619.0,2.0,23.0,269.545441,268.5,268.5,0.0,1.0,268.534180,268.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38234,0.186,249.0,8.0,283.0,5.0,246.0,22.0,523.0,3.0,244.0,...,795.0,4.0,34.0,269.923065,266.0,265.5,-0.5,0.0,266.008026,266.000000
38235,0.124,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,795.0,4.0,33.0,269.307678,265.5,265.5,0.0,1.0,265.544495,265.500001
38236,0.031,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,795.0,4.0,33.0,269.307678,265.5,265.5,0.0,1.0,265.541656,265.500001
38237,0.248,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,795.0,4.0,33.0,269.307678,265.5,265.0,-0.5,0.0,265.533051,265.500001


In [50]:
new_df.drop(new_df.columns[21:27], axis=1,inplace=True)
new_df

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level4_Bid_Price,level4_Bid_Volume,level4_Ask_Price,level4_Ask_Volume,level5_Bid_Price,level5_Bid_Volume,level5_Ask_Price,level5_Ask_Volume,Test Predictions,Actuals
0,0.062,257.0,6.0,296.0,5.0,256.0,1.0,306.0,2.0,255.0,...,253.0,1.0,441.0,5.0,252.0,5.0,583.0,2.0,276.183990,276.500000
1,0.062,257.0,6.0,281.0,5.0,256.0,1.0,296.0,5.0,255.0,...,253.0,1.0,348.0,4.0,252.0,5.0,583.0,2.0,268.955353,269.000001
2,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,252.0,5.0,348.0,4.0,251.0,3.0,583.0,2.0,268.477997,268.500000
3,0.186,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,252.0,5.0,583.0,2.0,251.0,3.0,619.0,2.0,268.564880,268.500000
4,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,255.0,...,253.0,1.0,583.0,2.0,252.0,5.0,619.0,2.0,268.534180,268.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38234,0.186,249.0,8.0,283.0,5.0,246.0,22.0,523.0,3.0,244.0,...,148.0,7.0,724.0,4.0,132.0,1.0,795.0,4.0,266.008026,266.000000
38235,0.124,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,148.0,7.0,724.0,4.0,132.0,1.0,795.0,4.0,265.544495,265.500001
38236,0.031,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,148.0,7.0,724.0,4.0,132.0,1.0,795.0,4.0,265.541656,265.500001
38237,0.248,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,148.0,15.0,724.0,4.0,132.0,1.0,795.0,4.0,265.533051,265.500001


In [52]:
new_df['PriceDiff'] = new_df['Test Predictions'].pct_change()
new_df

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level4_Bid_Volume,level4_Ask_Price,level4_Ask_Volume,level5_Bid_Price,level5_Bid_Volume,level5_Ask_Price,level5_Ask_Volume,Test Predictions,Actuals,PriceDiff
0,0.062,257.0,6.0,296.0,5.0,256.0,1.0,306.0,2.0,255.0,...,1.0,441.0,5.0,252.0,5.0,583.0,2.0,276.183990,276.500000,NaN
1,0.062,257.0,6.0,281.0,5.0,256.0,1.0,296.0,5.0,255.0,...,1.0,348.0,4.0,252.0,5.0,583.0,2.0,268.955353,269.000001,-0.026173
2,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,5.0,348.0,4.0,251.0,3.0,583.0,2.0,268.477997,268.500000,-0.001775
3,0.186,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,5.0,583.0,2.0,251.0,3.0,619.0,2.0,268.564880,268.500000,0.000324
4,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,255.0,...,1.0,583.0,2.0,252.0,5.0,619.0,2.0,268.534180,268.500000,-0.000114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38234,0.186,249.0,8.0,283.0,5.0,246.0,22.0,523.0,3.0,244.0,...,7.0,724.0,4.0,132.0,1.0,795.0,4.0,266.008026,266.000000,-0.000378
38235,0.124,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,7.0,724.0,4.0,132.0,1.0,795.0,4.0,265.544495,265.500001,-0.001743
38236,0.031,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,7.0,724.0,4.0,132.0,1.0,795.0,4.0,265.541656,265.500001,-0.000011
38237,0.248,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,15.0,724.0,4.0,132.0,1.0,795.0,4.0,265.533051,265.500001,-0.000032


In [56]:
#set policy: when the stock price decrease, HSBC whill buy the stock from clients; when the stock price incerese, HSBC will sell their stock to clients.
new_df['state'] = np.where(new_df['PriceDiff'] < -0.05, 'bid', np.where(new_df['PriceDiff'] > 0.05, 'ask', 0))
new_df

,Time,level1_Bid_Price,level1_Bid_Volume,level1_Ask_Price,level1_Ask_Volume,level2_Bid_Price,level2_Bid_Volume,level2_Ask_Price,level2_Ask_Volume,level3_Bid_Price,...,level4_Ask_Price,level4_Ask_Volume,level5_Bid_Price,level5_Bid_Volume,level5_Ask_Price,level5_Ask_Volume,Test Predictions,Actuals,PriceDiff,state
0,0.062,257.0,6.0,296.0,5.0,256.0,1.0,306.0,2.0,255.0,...,441.0,5.0,252.0,5.0,583.0,2.0,276.183990,276.500000,NaN,0
1,0.062,257.0,6.0,281.0,5.0,256.0,1.0,296.0,5.0,255.0,...,348.0,4.0,252.0,5.0,583.0,2.0,268.955353,269.000001,-0.026173,0
2,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,348.0,4.0,251.0,3.0,583.0,2.0,268.477997,268.500000,-0.001775,0
3,0.186,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,253.0,...,583.0,2.0,251.0,3.0,619.0,2.0,268.564880,268.500000,0.000324,0
4,0.031,257.0,6.0,280.0,5.0,256.0,1.0,281.0,5.0,255.0,...,583.0,2.0,252.0,5.0,619.0,2.0,268.534180,268.500000,-0.000114,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38234,0.186,249.0,8.0,283.0,5.0,246.0,22.0,523.0,3.0,244.0,...,724.0,4.0,132.0,1.0,795.0,4.0,266.008026,266.000000,-0.000378,0
38235,0.124,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,724.0,4.0,132.0,1.0,795.0,4.0,265.544495,265.500001,-0.001743,0
38236,0.031,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,724.0,4.0,132.0,1.0,795.0,4.0,265.541656,265.500001,-0.000011,0
38237,0.248,249.0,8.0,282.0,5.0,246.0,22.0,523.0,3.0,244.0,...,724.0,4.0,132.0,1.0,795.0,4.0,265.533051,265.500001,-0.000032,0


In [59]:
ask_count = (new_df['state'] == 'ask').sum()
ask_count

1095

In [65]:
#Set initial values
S = 100000 # Initial funding
N = 10000 #Initial Shares

for i, row in new_df.iterrows():
    if row['state'] == 'bid':
        S = S-row['level5_Bid_Price']*row['level5_Bid_Volume']
        N = N + row['level5_Bid_Volume']
    if row['state'] == 'ask':
        S = S+row['level1_Ask_Price']*row['level1_Ask_Volume']
        N = N - row['level1_Ask_Volume']
print(S,N)

738736.0 10701.0
